# Analyzing San Diego's Housing Auction Data

## Data Sources

The core analysis done in this report revolves around estimating a winning bid and estimating the potential sale price of a property. To estimate a winning bid for properties in the upcoming auction, we will need the following data:

<ul> 
    <li>Historical Auction Data</li>
    <li>Upcoming Auction Data</li>
    <li>Tax Assessor Parcel Data</li>
    <li>Foursquare Data</li>
    <li>San Diego Property Sales Data</li>
</ul>

To estimate the potential sale price of a property after the auction, we will need the following data:

<ul> 
    <li>Tax Assessor Parcel Data</li>
    <li>Foursquare Data</li>
    <li>San Diego Property Sales Data</li>
</ul>

The following sections will go into detail of how each data source will be used for our analysis.

### Historical Auction Data

The San Diego Treasurer-Tax Collector provides data on past auctions. This data contains information on which property went on auction, the opening bid and winning bid, along with other informational variables. Here is a sample of the data:

In [8]:
import pandas as pd

histauctdata = pd.read_csv('TotalSales.csv')
print(histauctdata.shape)
histauctdata.head()

(5235, 13)


,ID#,APN,Address,Property Description,Tax Rate Area,Land Value,Improvements,Assessed Value,Assessed Value Year,Sale Date,Opening Bid,Winning Bid,Notes
0,0001,1026011300,FALLBROOK CA,DOC60-56347 IN NEQ SEC 8-9-3W,75022,7010.00,0.00,7010.00,2019,3/4/2020,3000.0,12900.0,NaN
1,0019,1291807500,REGINA GLN VALLEY CENTER CA 92082,PAR 1,94075,132088.00,0.00,132088.00,2019,3/4/2020,17500.0,142100.0,NaN
2,0020,1292923600,11146 OLD CASTLE RD VALLEY CENTER CA 92082-5605,18.94 AC M/L IN PAR 1,94050,519312.00,382992.00,902304.00,2019,3/4/2020,434000.0,950100.0,Forfeited
3,0021,1293303400,30642 ROLLING HILLS DR VALLEY CENTER CA 92082-...,(EX DOCS53418REC70&amp;23787REC72)SWQ OF NEQ S...,94058,660572.00,30020.00,690592.00,2019,3/4/2020,332000.0,400100.0,Forfeited
4,0022,1293902900,CIRCLE R LN VALLEY CENTER CA 92082,PAR 4,94025,166252.00,0.00,166252.00,2019,3/4/2020,26700.0,115100.0,NaN


This data will be used (along with other data sources) to model winning bids.

### Upcoming Auction Data

The upcoming auction provides a workbook of propertie that are available for bid. This will be the catalog of potential investment for investors. A smaple of the data is provided below: 

In [9]:
auctiondata = pd.read_excel('3_17_2021_Auction_List_ImprovedProperty_20210215_2040.xls')
print(auctiondata.shape)  
auctiondata.head()

(63, 15)


,ID#,APN,Street Address,Tax Rate Area,Land Value,Improvements,Total Assessed Value,Assessed Value Year,Property Description,Your Max Bid,Opening Bid,Best Bid,Status,Close(PDT),Canceled
0,5,1023622000,5710 RAINBOW HEIGHTS RD FALLBROOK CA 92028-8844,93011,116469,88498,204967,2020,(EX RD)PAR A PER DOC89-510014 IN SEQ OF SEQ SE...,0,101000,0,-,2021-03-15 08:00:00,NaN
1,22,1211304300,4782 SLEEPING INDIAN RD FALLBROOK CA 92028-8875,75015,124409,348619,473028,2020,DOC59-6680 IN SWQ SEC 13-10-4W,0,232000,0,-,2021-03-15 08:00:00,NaN
2,23,1213223100,3816 CAZADOR LN FALLBROOK CA 92028-8880,75009,225195,225195,450390,2020,PAR 3,0,221000,0,-,2021-03-15 08:00:00,NaN
3,27,1233402300,4577 LA CANADA RD FALLBROOK CA 92028-9488,75013,103536,169812,273348,2020,POR LOT 58,0,134000,0,-,2021-03-15 08:00:00,NaN
4,34,1292110800,10079 W LILAC RD ESCONDIDO CA 92026-5308,94099,169793,318362,488155,2020,(EX ST) PAR 3,0,240000,0,-,2021-03-15 08:00:00,NaN


### Tax Assessor Parcel Data

The auction data alone does not provide enough information to accurately predict the winning bid for a property. It misses key information such as number of bedrooms, number of bathrooms, square footage, etc. These parameters are crucial to understanding what the different features of a property. Thankfully, the county tax assessor provides information on each property parcel in San Diego County. This data source contains all of the information needed to make a tax asseessment. Much of the information in this data source overlaps with our needs. Below is a sample of the data:

In [11]:
pip install simpledbf

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/72/dd/df/cdfb970a508ef9750ebb4b6e035a3410c3d62b5a6a91d2aafc
Successfully built simpledbf
Note: you may need to restart the kernel to use updated packages.


In [12]:
from simpledbf import Dbf5

dbf = Dbf5('PARCELS.dbf')
df = dbf.to_dataframe()

print(df.shape) 
print(df.columns)
df.head()

PyTables is not installed. No support for HDF output.
(1075406, 63)
Index(['APN', 'APN_8', 'PARCELID', 'OWN_NAME1', 'OWN_NAME2', 'OWN_NAME3',
       'FRACTINT', 'OWN_ADDR1', 'OWN_ADDR2', 'OWN_ADDR3', 'OWN_ADDR4',
       'OWN_ZIP', 'SITUS_JURI', 'SITUS_STRE', 'SITUS_SUFF', 'SITUS_POST',
       'SITUS_PRE_', 'SITUS_ADDR', 'SITUS_FRAC', 'SITUS_BUIL', 'SITUS_SUIT',
       'LEGLDESC', 'ASR_LAND', 'ASR_IMPR', 'ASR_TOTAL', 'DOCTYPE', 'DOCNMBR',
       'DOCDATE', 'ACREAGE', 'TAXSTAT', 'OWNEROCC', 'TRANUM', 'ASR_ZONE',
       'ASR_LANDUS', 'UNITQTY', 'SUBMAP', 'SUBNAME', 'NUCLEUS_ZO',
       'NUCLEUS_US', 'SITUS_COMM', 'YEAR_EFFEC', 'TOTAL_LVG_', 'BEDROOMS',
       'BATHS', 'ADDITION_A', 'GARAGE_CON', 'GARAGE_STA', 'CARPORT_ST', 'POOL',
       'PAR_VIEW', 'USABLE_SQ_', 'QUAL_CLASS', 'NUCLEUS_SI', 'NUCLEUS__1',
       'NUCLEUS__2', 'SITUS_ZIP', 'x_coord', 'y_coord', 'overlay_ju',
       'sub_type', 'multi', 'SHAPE_STAr', 'SHAPE_STLe'],
      dtype='object')


,APN,APN_8,PARCELID,OWN_NAME1,OWN_NAME2,OWN_NAME3,FRACTINT,OWN_ADDR1,OWN_ADDR2,OWN_ADDR3,...,NUCLEUS__1,NUCLEUS__2,SITUS_ZIP,x_coord,y_coord,overlay_ju,sub_type,multi,SHAPE_STAr,SHAPE_STLe
0,5911404900,59114049,13233,JOCIS LIVING TRUST 05-11-19,NaN,NaN,1.0,3451 MALITO DR,BONITA CA,NaN,...,0,NaN,91902,6.317739e+06,1.823133e+06,CN,1,N,2197.635742,262.298507
1,4982603900,49826039,13235,SCHAFFROTH EDWARD&GALE FAMILY TRUST 07-05-01,NaN,NaN,1.0,1640 CHASE LN #B,EL CAJON CA,NaN,...,0,NaN,92020-8306,6.350493e+06,1.860789e+06,CN,1,N,55587.956055,982.044216
2,4982604500,49826045,13236,REHUREK BARRY,NaN,NaN,1.0,P O BOX 1062,EL CAJON CA,NaN,...,0,NaN,92020-8270,6.350531e+06,1.861636e+06,CN,1,N,57767.315430,925.790174
3,6782511200,67825112,13238,PROMONTORY ASSOCIATES,PROMONTORY ASSOCIATES,NaN,4.0,C/O CARLETON MANAGEMENT INC,11440 W BERNARDO CT #390,SAN DIEGO CA,...,0,NaN,92127,6.306441e+06,1.952659e+06,SD,1,N,142397.666016,1565.520959
4,4775902300,47759023,13244,NGUYEN-LE LIVING TRUST 11-26-19,NaN,NaN,1.0,4857 EBONY RIDGE RD,SAN DIEGO CA,NaN,...,0,NaN,92105,6.304368e+06,1.847093e+06,SD,1,N,2768.697266,275.716807


This data will be used as features of a property in helping to determine the value of a property. Determining the value of a property will be crucial to estimating a winning bid and a the sale price of a home.

### Foursquare Data

In determining the value of a property, it is also wise to check the surroundings of the property. In real estate, "Location, location, location..." is often touted as one of the most important aspects of a property. Understanding where exactly a property is can change the value of a home drastically. Being in close proximity to schools, shopping, freeways, etc. is one of the most important aspects for people buying a home. The Foursquare data will helps us determine which venues are around a certain property. We can use this data to fill in several features of a property (alongside with the parcel data). Here is a sample of the Foursquare data:

In [20]:
import requests

CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [18]:
sd_lat = 32.7157
sd_long= -117.1611
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, sd_lat, sd_long, VERSION, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60316dc78e5f3a3826402105'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'With specials', 'key': 'specials'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Central San Diego',
  'headerFullLocation': 'Central San Diego, San Diego',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 140,
  'suggestedBounds': {'ne': {'lat': 32.7202000045, 'lng': -117.15576151471417},
   'sw': {'lat': 32.711199995499996, 'lng': -117.16643848528584}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '428d2880f964a520ac231fe3',
       'name': 'House of Blues San Diego',
       'location': {'address': '1055 5th Ave',
        'crossStreet': 'btwn Broadway & C St',
        'lat': 32.7164

The Foursquare features of a property will be used to estimate both a winning bid at auction and the potential sale price of a home.

### San Diego Property Sales Data

The final data source that we will use in this analysis will be San Diego's property sales data. This data will be used to determine the sale price of a property at any given time. Knowing the potential sale price of a home will be a key feature in estimating a winning bid. Note that for the past auction data, we will need to determine the potential sale price of a property at the time of the past auction. For example, if the auction took place in 2017, we will need to determine the property's value as of 2017 to help accurately model what a winning bid would have been. Also, given the upcoming auction, we will use the modeled sales price as a feature in determining the winning bid. Below is an example of the data:

In [19]:
condosales2020 = pd.read_csv('Condos sold 2020.csv')
print(condosales2020.shape)
condosales2020.head()

(2457, 22)


,Listing Pictures,MLS #,Property Type,PropSubT,Address,Postal Code,City,MLS Area,Status,Price,...,LivingArea,DOM,Close Price,Close Date,Beds Total,Baths Total,Sold Price Per SQFT,List Price,HOA Fee,Parking Total
0,http://media.crmls.org/mediaz/1DFA0E5F-1302-4B...,200037100,RESIDENTIAL,CONDO,2920 Briarwood F12,91902,SD,91902,Closed,"$285,000",...,599,51,"$285,000",12/4/2020,1,1.0,$475.79,"$280,000",$257,1
1,http://media.crmls.org/mediaz/F8AB874A-01F2-45...,200002110,RESIDENTIAL,CONDO,1234 Neptune,91911,SD,91911,Closed,"$399,000",...,1374,13,"$399,000",2/21/2020,3,2.0,$290.39,"$399,000",$0,4
2,//cdnparap150.paragonrels.com/ParagonImages/Pr...,200024679,RESIDENTIAL,CONDO,7504 Parkway Dr 208,91942,SD,91942,Closed,"$265,000",...,646,5,"$265,000",7/14/2020,1,1.0,$410.22,"$255,000",$272,1
3,http://media.crmls.org/mediaz/3F04B980-0B03-46...,200017840,RESIDENTIAL,CONDO,3514 Fairlomas Rd,91950,SD,91950,Closed,"$427,500",...,1400,19,"$427,500",5/27/2020,2,3.0,$305.36,"$409,999",$350,2
4,//cdnparap150.paragonrels.com/ParagonImages/Pr...,200008072,RESIDENTIAL,CONDO,1473 Gustavo St. B,92019,SD,92019,Closed,"$339,900",...,1188,44,"$339,900",4/1/2020,2,2.0,$286.11,"$339,900",$326,2


Furthermore, the sales data will help us create estimates of a property's sales prices to determine if an investment is worth it or not. It is not enough to win a bid, but an investor must also know how much a property can sell for. A rudimentary calculation for determining profit can be:

    Profit = Sale Price - Winning Bid Amount
    
Ideally, the sales price is large and the winning bid amount is small to maximize profit.